<a href="https://colab.research.google.com/github/vatsaaa/mtech/blob/main/semester_1/03_assignments/mfml/Assignment_01_Q01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python function to create a random matrix, but without using libraries

In [4]:
import random

In [5]:
def generate_random_matrix(rows: int, columns: int, decimal_allowed: bool=False):
    matrix = []

    for _ in range(rows):
      if decimal_allowed:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.choice([random.randint(100, 999), round(random.uniform(0, 1), 2)]) for _ in range(columns)]
      else:
        # Chose 100 and 990 so that the matrix printing is better to look at
        row = [random.randint(100, 999) for _ in range(columns)]

        matrix.append(row)

    return matrix

In [8]:
two_by_two = generate_random_matrix(2, 2, False)

two_by_two

[[334, 421], [116, 316]]

The above generated matrix is in a single line, that really does not look like a matrix. So, I create my own function to print a matrix that looks like a matrix.

In [7]:
def print_matrix(matrix):
    print("[", end="")
    for i, row in enumerate(matrix):
        print(row, end="")
        if i != len(matrix) - 1:
            print(",\n", end="")
    print("]")

In [9]:
print_matrix(two_by_two)

[[334, 421],
[116, 316]]


Now we get a two_by_one matrix, before creating the augmented matrix

In [10]:
two_by_one = generate_random_matrix(2, 1, False)

print_matrix(two_by_one)

[[346],
[277]]


Below function will create the augmented matrix required by Gaussian Elimination when

In [11]:
def construct_augmented_matrix(A, b):
    if len(A) != len(b):
        raise ValueError("Number of rows in A must be equal to the length of b")

    # Zip A and b to create the augmented matrix
    augmented_matrix = [row + [bi] for row, bi in zip(A, b)]

    return augmented_matrix

In [12]:
A = two_by_two
b = two_by_one

augmented_matrix_A_b = construct_augmented_matrix(A, b)

print_matrix(augmented_matrix_A_b)

[[334, 421, [346]],
[116, 316, [277]]]


Function to find row echelon form of a given matrix

In [18]:
from math import floor

def find_row_echelon_form(matrix):
    rows = len(matrix)
    columns = len(matrix[0])

    pivot = 0
    for row in range(rows):
        if pivot >= columns:
            break

        # When pivot is 0 we need a row to swap with the current row
        if matrix[row][pivot] == 0:
          for r in range(row + 1, rows):
            if matrix[r][pivot] != 0:
              matrix[row], matrix[r] = matrix[r], matrix[row]
              break
            else:
              pivot += 1

        # Now, when pivot position is not 0, do elementary row transformation
        # to ensure below the pivot we get zero
        for r in range(row + 1, rows):
          factor = floor(matrix[r][pivot] / matrix[row][pivot])
          for c in range(pivot, columns):
            matrix[r][c] = matrix[r][c] - factor * matrix[row][c]

        pivot += 1

    return matrix


In [ ]:
ref_A_b = find_row_echelon_form(augmented_matrix_A_b)

print_matrix(ref_A_b)